In [ ]:
import numpy as np
import pandas as pd
import arch
from arch import arch_model
from riskfolio import VaR_Hist
from riskfolio import CVaR_Hist
import seaborn as sns

In [ ]:
ret = pd.read_excel(r'return.xlsx')
ret.set_index('Date', inplace= True)

In [ ]:
#returns
#returns['Date'] = returns['Date'].astype('datetime64')
#returns.set_index('Date', inplace= True)

In [ ]:
ret

,vhm,vcb,fpt,agg,agr,asp,msb,nvl,ree
Date,,,,,,,,,
2020-12-24,-0.012665,-0.002054,-0.008734,-0.045068,-0.043616,-0.034412,0.017492,-0.015040,-0.016632
2020-12-25,0.014951,0.003079,0.010471,0.036556,0.047067,0.032790,0.067059,0.007550,0.006271
2020-12-28,0.001141,0.004090,0.005196,-0.015505,0.000000,-0.016252,0.013429,0.008979,-0.007321
2020-12-29,0.029215,0.012170,0.012017,-0.008716,-0.017387,0.006529,0.013243,0.011854,0.001049
2020-12-30,-0.001108,-0.013191,-0.008569,-0.010564,-0.018893,0.001621,-0.005280,-0.016333,0.012501
...,...,...,...,...,...,...,...,...,...
2023-03-27,-0.012320,0.022223,0.006341,0.000000,0.042966,0.005479,0.000000,0.065064,0.007380
2023-03-28,-0.008299,0.008753,0.001263,-0.001771,-0.007034,-0.001823,0.004107,-0.007905,0.000000
2023-03-29,0.000000,0.005432,-0.002528,-0.003552,0.022107,0.000000,0.000000,-0.003976,-0.011834


In [ ]:
ret.describe()

,vhm,vcb,fpt,agg,agr,asp,msb,nvl,ree
count,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000,564.000000
mean,-0.000397,0.000339,0.001210,0.000385,0.000170,-0.000117,0.000399,-0.001125,0.000857
std,0.021049,0.018322,0.019052,0.018938,0.034247,0.029956,0.023783,0.028062,0.025143
min,-0.072425,-0.072372,-0.072400,-0.097896,-0.077782,-0.112869,-0.072404,-0.072546,-0.072539
25%,-0.011177,-0.010132,-0.006542,-0.006951,-0.018229,-0.013979,-0.009394,-0.009975,-0.012028
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001247
75%,0.009593,0.009792,0.009695,0.006419,0.018302,0.013178,0.010809,0.009039,0.011904
max,0.067467,0.060625,0.067628,0.096159,0.067661,0.067663,0.067659,0.207897,0.067471


In [ ]:
train_data = ret.iloc[:500,:]
test_data = ret.iloc[500:,:]

In [ ]:
coeffs = []
cond_vol = []
std_resids = []
models = []
infor_aic = []
infor_bic = []
pred = []

In [ ]:
for asset in train_data.columns:
    model = arch_model(train_data[asset], mean='Constant', vol='EGARCH', p=1, o=0, q=1).fit(update_freq=0, disp='off')
    predict = model.forecast(horizon= len(test_data), method= 'simulation')
    coeffs.append(model.params)
    cond_vol.append(model.conditional_volatility)
    std_resids.append(model.resid / model.conditional_volatility)
    models.append(model)
    infor_aic.append(model.aic)
    infor_bic.append(model.bic)
    pred.append(predict.variance.values[-1])

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004252. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\arch\__future__\_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\arch\univariate\bas

In [ ]:
models

[                     Constant Mean - EGARCH Model Results                     
 Dep. Variable:                    vhm   R-squared:                       0.000
 Mean Model:             Constant Mean   Adj. R-squared:                  0.000
 Vol Model:                     EGARCH   Log-Likelihood:                1261.80
 Distribution:                  Normal   AIC:                          -2515.60
 Method:            Maximum Likelihood   BIC:                          -2498.75
                                         No. Observations:                  500
 Date:                Mon, Apr 24 2023   Df Residuals:                      499
 Time:                        00:43:41   Df Model:                            1
                                    Mean Model                                  
                   coef    std err          t      P>|t|        95.0% Conf. Int.
 -------------------------------------------------------------------------------
 mu         -9.9982e-04  2.034e-04   

In [ ]:
coeffs_df = pd.DataFrame(coeffs).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
cond_vol_df = pd.DataFrame(cond_vol).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
std_resids_df = pd.DataFrame(std_resids).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
pred_df = pd.DataFrame(np.sqrt(pred)).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
pred_df.set_index(test_data.index, inplace= True)
MSE = np.mean((pred_df - test_data)**2)
infor_df = pd.DataFrame({'AIC': infor_aic, 'BIC': infor_bic, 'MSE': MSE}).set_index(train_data.columns, inplace=False)

c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [ ]:
coeffs_df

,vhm,vcb,fpt,agg,agr,asp,msb,nvl,ree
mu,-0.001000,0.000226,0.001099,0.000590,0.000937,-0.000626,0.001290,0.000970,0.000739
omega,-0.854877,-1.744081,-0.392047,-0.342426,-0.862091,-1.930579,-0.532686,-0.435488,-0.387864
alpha[1],0.276639,0.271417,0.229617,0.094612,0.247913,0.334388,0.237505,0.357584,0.166114
beta[1],0.888769,0.780704,0.948314,0.955347,0.871608,0.721388,0.925999,0.935687,0.946346


In [ ]:
infor_df

,AIC,BIC,MSE
vhm,-2515.604749,-2498.746317,0.001042
vcb,-2592.115287,-2575.256855,0.000578
fpt,-2566.007802,-2549.149370,0.000554
agg,-2533.351580,-2516.493148,0.000542
agr,-1952.801969,-1935.943536,0.001666
asp,-2076.733212,-2059.874780,0.001196
msb,-2322.747933,-2305.889500,0.001100
nvl,-2404.466857,-2387.608424,0.003185
ree,-2285.694252,-2268.835820,0.000976


## Giá trị VaR và CVaR dự báo

In [ ]:
VaR_95 = []
CVaR_95 = []
VaR_99 = []
CVaR_99 = []

for i in pred_df.columns:
    cal_VaR_95 = VaR_Hist(pred_df[i], alpha= 0.05)
    cal_CVaR_95 = CVaR_Hist(pred_df[i], alpha= 0.05)
    cal_VaR_99 = VaR_Hist(pred_df[i], alpha= 0.01)
    cal_CVaR_99 = CVaR_Hist(pred_df[i], alpha= 0.01)
    VaR_95.append(cal_VaR_95)
    CVaR_95.append(cal_CVaR_95)
    VaR_99.append(cal_VaR_99)
    CVaR_99.append(cal_VaR_99)

VaR_df = pd.DataFrame({'VaR 95%': VaR_95,
                       'VaR 99%': VaR_99,
                       'CVaR 95%': CVaR_95,
                       'CVaR 99%': CVaR_99}).set_axis(train_data.columns, inplace=False)
VaR_df

,VaR 95%,VaR 99%,CVaR 95%,CVaR 99%
vhm,-0.021917,-0.021755,-0.021846,-0.021755
vcb,-0.016839,-0.014333,-0.015361,-0.014333
fpt,-0.016149,-0.015118,-0.015509,-0.015118
agg,-0.018254,-0.017778,-0.017956,-0.017778
agr,-0.033572,-0.032181,-0.032675,-0.032181
asp,-0.029840,-0.026727,-0.028081,-0.026727
msb,-0.024359,-0.023308,-0.023783,-0.023308
nvl,-0.037428,-0.037166,-0.037266,-0.037166
ree,-0.023168,-0.022392,-0.022734,-0.022392


In [ ]:
# Tính trọng số đều cho các mã
weights = np.array([1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9, 1/9])

# Tính VaR và CVaR của danh mục đầu tư
portfolio_VaR_95 = np.sum(weights * VaR_95)
portfolio_VaR_99 = np.sum(weights * VaR_99)
portfolio_CVaR_95 = np.sum(weights * CVaR_95)
portfolio_CVaR_99 = np.sum(weights * CVaR_99)

risk = pd.DataFrame(data= [portfolio_VaR_95, portfolio_VaR_99, portfolio_CVaR_95, portfolio_CVaR_99],
                    index= ['VaR 95%', 'VaR 99%', 'CVaR 95%', 'CVaR 99%'], columns= ['Portfolio'])
risk

,Portfolio
VaR 95%,-0.024614
VaR 99%,-0.023417
CVaR 95%,-0.023912
CVaR 99%,-0.023417


In [ ]:
#from scipy.optimize import minimize

In [ ]:
#def objective_function(weights, VaR_99, CVaR_99):
   # portfolio_VaR = np.sum(weights * VaR_99)
   # portfolio_CVaR = np.sum(weights * CVaR_99)
   # return portfolio_CVaR - portfolio_VaR

In [ ]:
# Thêm ràng buộc và hạn chế để đảm bảo rằng tổng trọng số bằng 1 và các trọng số không âm
#def constraint1(weights):
    #return np.sum(weights) - 1

In [ ]:
#constraints = [{'type': 'eq', 'fun': constraint1}]
#bounds = [(0, 1)] * len(VaR)

In [ ]:
#optimized = minimize(objective_function,
                   #  x0 = [1.0/len(train_data.columns)]*len(train_data.columns),
                   #  args=(VaR_95, CVaR_95),
                   #  method='SLSQP',
                   #  bounds=bounds,
                   #  constraints=constraints)

In [ ]:
#new_weight = pd.DataFrame(optimized.x, columns= ['weight']).transpose().set_axis(train_data.columns, axis='columns', inplace=False)
#new_weight.applymap('{:.2%}'.format)